## import pandas as pd
import re

In [3]:
# 讀取cliwc.dicx，建為DataFrame
def loadCliwctoDf():
    with open('LIWC2015 Dictionary - Chinese (Traditional) (v1.5).dicx', encoding='utf-8') as cliwcD:
        lines = cliwcD.readlines()
        i=0
        for line in lines:
            if i == 0:
                l = line.split('\n')[0].split(',')
                df = pd.DataFrame(columns= l)
            if i > 0:
                l = line.split('\n')[0].split(',')
                l = [ 0 if i=='' else i  for i in l ]
                l[0] = l[0].replace('"','') 
                l[0] = l[0].replace(')','\)')
                l[0] = l[0].replace('(','\(')
                l[0] = l[0].replace('+','\+')
                l[0] = l[0].replace('*','\*')
                df.loc[len(df.index)] = l
            i+=1
        return df

In [4]:
# 讀取out.txt，輸出以列為元素的list
def loadOut():
    with open('out.txt', encoding='utf-8') as out:
        lines = out.readlines()
    return lines

In [5]:
df = loadCliwctoDf()           # 把cliwc建好dataFrame，名稱為df。
EntryTup = tuple(df['Entry'])  # 將關鍵字的欄位取出為series，名稱為Entry
lines = loadOut()              # 把out.txt讀出，放入變數lines。(list型態，元素為字串(待轉為字典))
articles = None                #將最後結果articles(DataFrame)先建好，待以迴圈將資料從lines取出，解析完之後一列一列塞資料進artcles


In [60]:
#迴圈開始
c = 0 #計次器: 代表跑到第幾筆資料
for line in lines:
    # 將字串從lines取出，轉型為字典，命名為dic
    dic = eval(line)
    # 幫字典新增七個key，分別為'留言推'、'留言噓'、'留言→'、'年'、'月'、'日'、'時'
    dic['留言推'], dic['留言噓'], dic['留言→'],dic['年'],dic['月'],dic['日'],dic['時'] = 0,0,0,0,0,0,0
    time = dic['日期']
    timepattern='(\S*)\s*(\S*)\s*(\d*)\s*(\d*:\d*:\d*)\s*(\d*)'
    date = re.findall(timepattern, time)
    dic['年'],dic['月'],dic['日'],dic['時'] = date[0][-1],date[0][1],date[0][2],date[0][3]
    #計算留言的推數、噓數、箭頭數。並將文章留言放入str1，文章內容放入str2
    message = '' #留言
    content = dic['內容']
    for tup in dic['留言']:
        if tup[0] == '推':
            dic['留言推'] += 1
        elif tup[0] == '噓':
            dic['留言噓'] += 1
        elif tup[0] == '→':
            dic['留言→'] += 1
        else:
            continue
        message += tup[2] #留言內容放到message
        
    # 開始解析message(留言)和content(文章內容)
    # 先解析message
    count1 = [] #建立list，當有關鍵字Entry被解析到時，就會在該Entry的位置計數
    mask1 = []  #建立遮罩list，當有關鍵字Entry被解析到時，就會在該Entry的位置給True，否則False
    for i in range(len(EntryTup)):
        l = [len(re.findall(r'{}'.format(EntryTup[i]), message))]
        mask1 += [True] if l!=[0] else [False]
        count1 += l
    # 把關鍵字位置計數的list轉為series，並只取出計數大於0的部分。
    count1 = pd.Series(count1)
    count1 = count1[count1>0]
    # 利用遮罩，把剛剛有解析到的每個關鍵字所對應到的liwc特徵加權，從df中取出，型態為DataFrame
    parse1 = df[mask1].loc[:,'function':].astype(int)
    # count為計數X關鍵字的向量，parse為關鍵字X特徵的矩陣。count和parse取dot，得計數X特徵的向量，再轉為DataFrame，命名為message_parse
    message_parse = count1.dot(parse1)
    message_parse = pd.DataFrame(message_parse).transpose()
    # 把DataFrame的column中的特徵名稱，加上'message_'
    message_parse.columns = 'message_'+ message_parse.columns
    # 再解析str2
    count2 = [] #建立list，當有關鍵字Entry被解析到時，就會在該Entry的位置計數
    mask2 = []  #建立遮罩list，當有關鍵字Entry被解析到時，就會在該Entry的位置給True，否則False
    for i in range(len(EntryTup)):
        l = [len(re.findall(r'{}'.format(EntryTup[i]), content))]
        mask2 += [True] if l!=[0] else [False]
        count2 += l
    # 把關鍵字位置計數的list轉為series，並只取出計數大於0的部分。
    count2 = pd.Series(count2)
    count2 = count2[count2>0]
    # 利用遮罩，把剛剛有解析到的每個關鍵字所對應到的liwc特徵加權，從df中取出，型態為DataFrame
    parse2 = df[mask2].loc[:,'function':].astype(int)
    # count為計數X關鍵字的向量，parse為關鍵字X特徵的矩陣。count和parse取dot，得計數X特徵的向量，再轉為DataFrame，命名為message_parse
    content_parse = count2.dot(parse2)
    content_parse = pd.DataFrame(content_parse).transpose()
    # 把DataFrame的column中的特徵名稱，加上'content_'
    content_parse.columns = 'content_'+ content_parse.columns
    # 把文章的日期、作者、標題取出，並與content_parse、message_parse合併。得到單筆article資料
    article_info = pd.DataFrame([dic['年'],dic['月'],dic['日'],dic['時'],dic['作者'],dic['標題'],dic['留言推'],dic['留言噓'],dic['留言→']],index=['年','月','日','時','作者','標題','推數','噓數','→數']).transpose()
    article = pd.concat([article_info,content_parse,message_parse],axis=1)
    articles = pd.concat([articles ,article],axis=0)
    # debug欄位
    c+=1
    print('已塞入第 ',c,'筆資料')

已塞入第  1 筆資料
已塞入第  2 筆資料
已塞入第  3 筆資料
已塞入第  4 筆資料
已塞入第  5 筆資料
已塞入第  6 筆資料
已塞入第  7 筆資料
已塞入第  8 筆資料
已塞入第  9 筆資料
已塞入第  10 筆資料
已塞入第  11 筆資料
已塞入第  12 筆資料
已塞入第  13 筆資料
已塞入第  14 筆資料
已塞入第  15 筆資料
已塞入第  16 筆資料
已塞入第  17 筆資料


KeyboardInterrupt: 

In [63]:
articles

,年,月,日,時,作者,標題,推數,噓數,→數,content_function,...,message_home,message_money,message_relig,message_death,message_informal,message_swear,message_netspeak,message_assent,message_nonflu,message_filler
0,2022,Sep,29,21:58:38,YAYA6655 (YAYA),Re: [心得] 2022賠錢賣屁股總結,7,2,13,140,...,3,19,2,1,28,1,10,17,5,2
0,2022,Sep,29,21:59:05,cutiehoneyzu (McHilo),Re: [請益] 目前美元指數下跌-1.32%，該進場了？,22,1,17,29,...,0,44,2,0,43,0,7,21,4,15
0,2022,Sep,29,21:59:29,a30030089 (軒),[心得] 盤後資料整理(Excel格式),1,0,0,112,...,0,0,0,0,0,0,0,0,0,0
0,2022,Sep,29,22:11:49,ArthurCurry (AQUAMAN),[情報] 合一:本公司舉辦111年第三季線上法說會,18,3,5,65,...,1,0,15,0,17,0,2,10,5,2
0,2022,Sep,29,22:24:23,randy225 (資方打手),[新聞] 快訊／美股大挫！四大指數皆跌 道瓊指,138,9,53,96,...,10,41,23,11,235,11,84,128,20,24
0,2022,Sep,29,22:58:43,dsrte (把握),Re: [新聞] 快訊／美股大挫！四大指數皆跌 道瓊指,0,0,0,208,...,0,0,0,0,0,0,0,0,0,0
0,2022,Sep,29,23:01:12,chinaeatshit (我愛台灣！中國吃屎！！),Re: [新聞] 快訊／美股大挫！四大指數皆跌 道瓊指,0,0,0,145,...,0,0,0,0,0,0,0,0,0,0
0,2022,Sep,29,23:02:31,offeree (offeree),[情報] 9/29全市場當沖虧損最多的股票,0,0,0,48,...,0,0,0,0,0,0,0,0,0,0
0,2022,Sep,29,18:48:27,chaming (無與倫比的熱情),[情報] 尚凡(5278)第2屆PPT鄉民法說會+20週年慶,280,1,27,93,...,5,28,9,7,373,10,282,80,12,5
0,2022,Sep,29,14:01:17,vendan5566 (阿文),[閒聊] 2022/09/29 盤後閒聊,821,69,602,26,...,64,279,106,108,1674,43,632,835,168,172


In [32]:
articles.index

RangeIndex(start=0, stop=1, step=1)

In [71]:
articles.to_csv('PttStock_CliwcParsed.csv',encoding='utf_8_sig',index=False)